In [ ]:
import pandas as pd
import numpy as np
#from pandas_datareader import data, wb
import datetime
import pprint

import sys,os
sys.path.append('..')
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
from backtester.swarms.swarm import Swarm
from backtester.exoinfo import EXOInfo
from exobuilder.data.exostorage import EXOStorage
from IPython.core.display import display, HTML

In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Loading global setting for MongoDB etc.
from scripts.settings import *

try:
    from scripts.settings_local import *
except:
    pass

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)
pp = pprint.PrettyPrinter(indent=4)

exo_list = storage.exo_list()
pp.pprint(exo_list)

In [ ]:
exo_series, exo_dict = storage.load_series('ES_CallSpread')
#exo_series, exo_dict = storage.load_series('ZN_CallSpread')
exo_series = exo_series[exo_series.columns[0]]

In [ ]:
figsize(20,10)
exo_series.plot()
exo_series.rolling(20).min().plot()

## Renko calculation

## High/Low of renko bricks is only needed for visual representation

In [ ]:
box_start = 0
box_start_idx = None

box_end = 0
box_quantity = 0

temp_l = []

# Simple renko algorihtm

df = pd.DataFrame()
df['close'] = exo_series

atr_period = 60

O = L = H = C = df['close']
period = atr_period

atr = ATR(H, L, C, period)

atr_multiplier = 4

for i in range(len(df)):
    box_size = (atr/atr_multiplier).shift(1)[i]
    
    if box_start == 0:
        box_start = df.close[i]

    else:
        box_start = box_start
        price_move = df.close[i] - box_start
        
        # First of all we need to set box size. 
        # Then calculate price movement. 
        # If price movement is more or equal than box size - renko bar(or bars) will be added
        
        if np.abs(price_move) >= box_size:
            
            # After we calculate box_quantity(price move divided by box size)
            # This number defines how much renko bars will be registred
            box_quantity = np.int32(np.floor(np.abs(price_move / box_size)))
            box_index = df.close.index[i]

            for b in range(int(box_quantity)):
            # Let say, index is 2015-01-01, box_start = 100, box_quantity = 3, box size = 10, price move > 0
            # So renko bar 1 will have next parameters - 
            # 1)index 2015-01-01
            # 2)open = 100
            # 3)close = 110(box_start + box_size)
            # 4)type = up
            
            # Next renko bar will have next parameters -
            # 1)index 2015-01-01
            # 2)open = 110(previous renko bar close)
            # 3)close = 120(open + box_size)
            # 4)type = up
            
            # And so on..
            
            # After all we adding renko bars dict to list and convert it to DF
            
                if price_move > 0:
                    if box_end == 0:
                        d = {'date': box_index, 'open': box_start, 'close': box_start + box_size, 'type': 'up'}
                        box_end = d['close']
                        temp_l.append(d)
                        
                    else:
                        d = {'date': box_index, 'open': box_end, 'close': box_end + box_size,
                            'type': 'up'}
                        
                        box_end = d['close']
                        temp_l.append(d)

                if price_move < 0:
                    if box_end == 0:
                        d = {'date': box_index, 'open': box_start, 'close': box_start - box_size, 'type': 'down'}
                        box_end = d['close']
                        temp_l.append(d)
                        
                    else:           
                        d = {'date': box_index, 'open': box_end, 'close': box_end - box_size, 
                             'type': 'down'}
                        
                        box_end = d['close']
                        temp_l.append(d)
                        
            box_start = df.close[i]
            
renko_df = pd.DataFrame(temp_l)

del temp_l

In [ ]:
bb_period = 20
bb_k = 2
renko_df['bb_central_line'] = renko_df.close.rolling(bb_period).mean()
renko_df['bb_upperband'] = renko_df.bb_central_line + (bb_k * renko_df.bb_central_line.rolling(bb_period).std())
renko_df['bb_lowerband'] = renko_df.bb_central_line - (bb_k * renko_df.bb_central_line.rolling(bb_period).std())

renko_df['bb_%b'] = (renko_df.close - renko_df['bb_lowerband']) / (renko_df['bb_upperband'] - renko_df['bb_lowerband'])
renko_df['bb_width'] = (renko_df['bb_upperband'] - renko_df['bb_lowerband'])

In [ ]:
renko_df.tail()

In [ ]:
renko_df.close.plot()
renko_df.bb_lowerband.plot()
renko_df.bb_upperband.plot()

In [ ]:
renko_df.close.plot()
((renko_df.close - renko_df.bb_upperband).rolling(5).mean()).plot(secondary_y=True)
(renko_df.close - renko_df.bb_upperband).plot(secondary_y=True)

In [ ]:
renko_df.close.plot()
(bb_width_pctrank).plot(secondary_y=True)
#(renko_df.close - renko_df.bb_upperband).plot(secondary_y=True)

In [ ]:
renko_df['up_trend'] = (((renko_df.close - renko_df.bb_upperband).rolling(5).median()) <= (renko_df.close - renko_df.bb_upperband))
renko_df['down_trend'] = (renko_df.bb_lowerband -  renko_df.close).rolling(5).median()  <= (renko_df.bb_lowerband -  renko_df.close)

bb_width_pctrank = renko_df.bb_width.rank(pct=True)
bb_pctb_pctrank = renko_df['bb_%b'].rank(pct=True)

renko_df['bb_width_pctrank_less_20'] = bb_width_pctrank <= 0.2
renko_df['bb_width_pctrank_less_10'] = bb_width_pctrank <= 0.1

renko_df['bb_width_pctrank_more_80'] = bb_width_pctrank >= 0.8
renko_df['bb_width_pctrank_more_90'] = bb_width_pctrank <= 0.9

renko_df['bb_width_pctrank_more_50'] = bb_width_pctrank >= 0.5
renko_df['bb_width_pctrank_less_50'] = bb_width_pctrank <= 0.5

renko_df['bb_pctb_pctrank_less_20'] = bb_pctb_pctrank <= 0.2
renko_df['bb_pctb_pctrank_less_10'] = bb_pctb_pctrank <= 0.1

renko_df['bb_pctb_pctrank_more_80'] = bb_pctb_pctrank >= 0.8
renko_df['bb_pctb_pctrank_more_90'] = bb_pctb_pctrank <= 0.9

renko_df['bb_pctb_pctrank_more_50'] = bb_pctb_pctrank >= 0.5
renko_df['bb_pctb_pctrank_less_50'] = bb_pctb_pctrank <= 0.5

In [ ]:
renko_df['entry_signal']  = (renko_df.bb_pctb_pctrank_less_10)#.plot()#.reindex(renko_df.date)).plot()

In [ ]:
df = df.join(renko_df.set_index('date')[['up_trend','down_trend','bb_width_pctrank_less_20','bb_width_pctrank_less_10', 
                                    'bb_width_pctrank_less_50', 'bb_width_pctrank_more_80', 'bb_width_pctrank_more_90', 
                                   'bb_width_pctrank_more_50', 'bb_pctb_pctrank_less_20', 'bb_pctb_pctrank_less_10', 
                                    'bb_pctb_pctrank_less_50', 'bb_pctb_pctrank_more_80', 'bb_pctb_pctrank_more_90',
                                   'bb_pctb_pctrank_more_50']])
df = df.fillna(False)

In [ ]:
df.close.plot()
df[df.columns[2]].plot(secondary_y=True)
#df.crossup.plot(secondary_y=True)
axhline(1.2)

In [ ]:
exo_series.plot()
exo_series.rolling(20).min().plot()
(exo_series > exo_series.rolling(20).min()+100).plot(secondary_y=True)
axhline(2)

In [ ]:
(exo_series > exo_series.rolling(20).min()+100).rolling(10).sum().plot()